In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///Users/runner/miniforge3/conda-bld/scikit-learn_1716489783876/work/dist/scikit_learn-1.5.0-cp310-cp310-macosx_11_0_arm64.whl#sha256=ab7c53589e5b1e0f9177283518952e043dc01f443b0ba1793babf09ce8b9f181


In [2]:
!python -V

Python 3.10.13


In [21]:
import pickle
import pandas as pd
import os

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
import numpy as np
std_minutes = np.std(y_pred, ddof=1) 
print(f"Standard deviation of predicted duration: {std_minutes:.3f} minutes")

Standard deviation of predicted duration: 6.247 minutes


In [17]:
year = 2023
month = 3   
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['preds'] = y_pred

In [18]:
df_result = pd.DataFrame({
         "ride_id": df["ride_id"],
         "predicted_duration": y_pred
     })

In [22]:
output_file = 'predictions.result.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
file_mb = os.path.getsize(output_file) / 1024**2
print(f"local parquet file   : {file_mb:,.2f} MB")

local parquet file   : 65.46 MB
